In [ ]:
import numpy as np
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import spsolve

In [ ]:
def get_indexs(i: int, j: int, n: int) -> int:
    """Returns the indices of the vector $u$."""
    nodes_per_row = 2**n + 1

    return i * nodes_per_row + j

In [ ]:
def vec_b(n: int) -> np.floating:
    """Returns the value of the right-hand side vector $b$."""
    h = np.float_power(2, -n)
    nodes_per_row = 2**n + 1

    def inner_f(x: np.floating, y: np.floating) -> np.floating:
        return (
            12 * np.power(x, 2) * np.power(y, 5)
            + 17 * (x**2 + y**2) * np.sin(x * y)
            + 20 * np.power(x, 4) * np.power(y, 3)
        ) * h**2

    b_ls = np.zeros(nodes_per_row**2)

    for i in range(nodes_per_row):
        for j in range(nodes_per_row):
            idx = get_indexs(i, j, n)
            if i == 0 or j == 0:
                b_ls[idx] = 0
            elif i == nodes_per_row - 1:
                y = j * h
                b_ls[idx] = np.power(y, 5) - 17 * np.sin(y)
            elif j == nodes_per_row - 1:
                x = i * h
                b_ls[idx] = np.power(x, 4) - 17 * np.sin(x)
            else:
                x = i * h
                y = j * h
                b_ls[idx] = inner_f(x, y)

    b_ls = np.reshape(b_ls, (nodes_per_row**2, 1))

    if b_ls.shape != (nodes_per_row**2, 1):
        shape_err = ValueError("The shape of b is not correct.")
        raise shape_err
    return b_ls

Create the matrix in sparse format

In [ ]:
def mat_A(n: int) -> coo_matrix:
    """Returns the value of the matrix $A$."""
    nodes_per_row = 2**n + 1
    row = np.array([])
    col = np.array([])
    A_ls = np.array([])

    for i in range(nodes_per_row):
        for j in range(nodes_per_row):
            idx = get_indexs(i, j, n)
            if i == 0 or j == 0 or i == nodes_per_row - 1 or j == nodes_per_row - 1:
                row = np.append(row, idx)
                col = np.append(col, idx)
                A_ls = np.append(A_ls, 1)
            else:
                row = np.append(row, idx)
                col = np.append(col, idx)
                A_ls = np.append(A_ls, -4)
                for idx_ in [
                    get_indexs(i - 1, j, n),
                    get_indexs(i + 1, j, n),
                    get_indexs(i, j - 1, n),
                    get_indexs(i, j + 1, n),
                ]:
                    row = np.append(row, idx)
                    col = np.append(col, idx_)
                    A_ls = np.append(A_ls, 1)

    A = coo_matrix((A_ls, (row, col)), shape=(nodes_per_row**2, nodes_per_row**2))
    if A.shape != (nodes_per_row**2, nodes_per_row**2):
        shape_err = ValueError("The shape of A is not correct.")
        raise shape_err
    return A

In [ ]:
def vec_u(n: int) -> np.ndarray:
    """Returns the value of the real solution $u$."""
    nodes_per_row = 2**n + 1
    u_ls = np.zeros(nodes_per_row**2)
    h = np.float_power(2, -n)

    for i in range(nodes_per_row):
        for j in range(nodes_per_row):
            idx = get_indexs(i, j, n)
            x = i * h
            y = j * h
            u_ls[idx] = np.power(x, 4) * np.power(y, 5) - 17 * np.sin(x * y)

    return u_ls

In [ ]:
norm_inf_ls = []
norm_2_ls = []

for n in range(2, 9):
    A = mat_A(n).tocsc()
    b = vec_b(n)
    u = vec_u(n)
    u_h = spsolve(A, b)

    norm_inf = np.max(np.abs(u - u_h))
    display(f"n={n}, norm_inf={norm_inf}")
    norm_inf_ls.append(norm_inf)

    norm_2 = np.linalg.norm(u - u_h, 2)
    display(f"n={n}, norm_2={norm_2}")
    norm_2_ls.append(norm_2)

order_inf = np.log(norm_inf_ls[-2] / norm_inf_ls[-1]) / np.log(2)
display(f"order of convergence of inf norm: {order_inf}")

order_2 = np.log(norm_2_ls[-2] / norm_2_ls[-1]) / np.log(2)
display(f"order of convergence of 2 norm: {order_2}")